# Lesson 1: Your first agent with Amazon Bedrock

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


## Start of the lesson

In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': 'eed37cc2-fefc-4df0-aa7c-35fdb07d9924',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 07 Aug 2025 00:35:19 GMT',
   'content-type': 'application/json',
   'content-length': '624',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'eed37cc2-fefc-4df0-aa7c-35fdb07d9924',
   'x-amz-apigw-id': 'O6MvJFibvHcEbLg=',
   'x-amzn-trace-id': 'Root=1-6893f4c6-0ff36629782b833b3a23b018'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:891377287105:agent/YFXT4EDMYQ',
  'agentId': 'YFXT4EDMYQ',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::891377287105:role/c135316a3428926l11136487t1w8913772-BedrockAgentRole-zvNAeyWTXW7d',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 8, 7, 0, 35, 19, 42852, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a fr

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': '01b48bf9-3a3a-468f-b44f-53a85f0b11b6',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 07 Aug 2025 00:39:10 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '01b48bf9-3a3a-468f-b44f-53a85f0b11b6',
   'x-amz-apigw-id': 'O6NTUEK_vHcEncA=',
   'x-amzn-trace-id': 'Root=1-6893f5ae-249a553717fdbb3810dfc7b7'},
  'RetryAttempts': 0},
 'agentId': 'YFXT4EDMYQ',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 8, 7, 0, 39, 10, 736396, tzinfo=tzlocal())}

In [10]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [12]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [13]:
import uuid

In [14]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [15]:
event_stream = invoke_agent_response["completion"]

In [16]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'PFBC3DJOKU', 'agentId': 'YFXT4EDMYQ', 'agentVersion': '1', 'sessionId': '9db7229b-6dc8-4408-975b-c536cc19576f', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are an advanced AI agent acting as a front line customer support agent. You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>       ...     </parameters>   </invoke> </function_calls> Here are the functions available: <functions>    </functions> You will ALWAYS follow the below guidelines when you are answering a question: <guidelines> - Think through the user\'s question, extract all data from the questi

In [17]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [18]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's see how I can assist you with returning the mug you
  purchased. I will need to gather some more information to process
  the return properly.

Agent's thought process:
  To process your return, I will need some more details about your
  order. Could you please provide the order ID or the date you made
  the purchase?

Observation:
  Type: FINISH

Final response:
  I apologize, but I seem to be having trouble retrieving the details
  of your order. Could you please provide me with the order ID or the
  date you made the purchase? That will help me look up the details
  and process your return request.


Session ID: 75d93a2d-8fd1-464b-9843-aaa8ce02c85c
